In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sif_multinput import KNN

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import display
from sklearn.preprocessing import StandardScaler

import plotly.graph_objects as go
pd.options.plotting.backend = "plotly"

In [2]:
# Data Preparation
df = pd.read_csv("insurance.csv")
df['is_female'] = df['sex'].apply(lambda x: 1 if x == 'female' else 0)
df['is_smoker'] = df['sex'].apply(lambda x: 1 if x == 'yes' else 0)
df = df.drop(columns=['sex', 'smoker', 'region'])
display(df)

df_train = df[:-300]
df_test = df[-300:]

scaler_input = StandardScaler()
scaler_target = StandardScaler()

U_train = scaler_input.fit_transform(df_train.drop(columns=['charges']).to_numpy())
Y_train = scaler_target.fit_transform(df_train[['charges']].to_numpy())

U_test = scaler_input.transform(df_test.drop(columns=['charges']).to_numpy())
Y_test = scaler_target.transform(df_test[['charges']].to_numpy())

,age,bmi,children,charges,is_female,is_smoker
0,19,27.900,0,16884.92400,1,0
1,18,33.770,1,1725.55230,0,0
2,28,33.000,3,4449.46200,0,0
3,33,22.705,0,21984.47061,0,0
4,32,28.880,0,3866.85520,0,0
...,...,...,...,...,...,...
1333,50,30.970,3,10600.54830,0,0
1334,18,31.920,0,2205.98080,1,0
1335,18,36.850,0,1629.83350,1,0
1336,21,25.800,0,2007.94500,1,0


In [4]:
# Create two identical KNN's that will be trained differently
hidden_layer = 50
knn_sif = KNN(nu=U_train.shape[1], ny=1, nl=hidden_layer, neuron='logistic')
knn_ekf = KNN(nu=U_train.shape[1], ny=1, nl=hidden_layer, neuron='logistic')
knn_sgd = KNN(nu=U_train.shape[1], ny=1, nl=hidden_layer, neuron='logistic')
nepochs = 100

print("----Neural Network Structure----")
print(f"Input --> {hidden_layer}neurons --> 1 neuron --> Output")
print(f"epochs: {nepochs}")
print("batch_size: 1")
print("")

# Train
mu = 0.00001
knn_sif.train(nepochs=nepochs, U=U_train, Y=Y_train, method='sif', P=(mu/0.01), Q=mu*0.0001, R=(mu/0.1))
knn_ekf.train(nepochs=nepochs, U=U_train, Y=Y_train, method='ekf', P=(mu/0.01), Q=mu*0.0001, R=(mu/0.1))
knn_sgd.train(nepochs=nepochs, U=U_train, Y=Y_train, method='sgd', step=0.1)

ekf_pred = [knn_ekf.feedforward(U_test[i,:])[0] for i in range(len(U_test))]
esif_pred = [knn_sif.feedforward(U_test[i,:])[0] for i in range(len(U_test))]
sgd_pred = [knn_sgd.feedforward(U_test[i,:])[0] for i in range(len(U_test))]

----Neural Network Structure----
Input --> 50neurons --> 1 neuron --> Output
epochs: 100
batch_size: 1



/Users/onur/Desktop/Github/SIF/sif_multinput.py:57: RuntimeWarning: overflow encountered in exp
  self.sig = lambda V: (1 + np.exp(-V))**-1
/Users/onur/miniforge3/lib/python3.9/site-packages/numpy/core/_methods.py:179: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims, where=where)
/Users/onur/Desktop/Github/SIF/sif_multinput.py:80: RuntimeWarning: overflow encountered in square
  self.compute_rms = lambda U, Y: np.sqrt(np.mean(np.square(Y - self.feedforward(U))))
/Users/onur/Desktop/Github/SIF/sif_multinput.py:359: RuntimeWarning: invalid value encountered in multiply
  D = (e.dot(self.W[1][:, :-1])*self.dsig(l)).flatten()


In [5]:
# Visualize
df_result = pd.DataFrame()

df_result['EKF-NN'] = scaler_target.inverse_transform(np.array(ekf_pred).reshape(-1,1)).ravel()
df_result['ESIF-NN'] = scaler_target.inverse_transform(np.array(esif_pred).reshape(-1,1))
df_result['SGD-NN'] = scaler_target.inverse_transform(np.array(sgd_pred).reshape(-1,1))
df_result['Actual'] = df_test['charges'].values
df_result.plot()



In [6]:
df_rms = pd.DataFrame()
df_rms['SGD_RMS'] = knn_sgd.RMS
df_rms["SIF_RMS"] = knn_sif.RMS
df_rms["EKF_RMS"] = knn_ekf.RMS
fig = df_rms.plot()
fig.show()